In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATA_DIR = 'nyt-data'

STATE = 'California'
COUNTY = 'Orange'


def calculate_daily_stats(df):
    df = df.set_index('date')
    df['daily_cases'] = df['cases'].diff().fillna(df['cases'])
    df['daily_deaths'] = df['deaths'].diff().fillna(df['deaths'])
    return df


country_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us.csv')),
    ],
    join='inner',
)
country_df = calculate_daily_stats(country_df)
state_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-states.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-states.csv')),
    ],
    join='inner',
)
state_df = calculate_daily_stats(state_df[state_df['state'] == STATE])
county_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-counties.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-counties.csv')),
    ],
    join='inner',
)
county_df = calculate_daily_stats(county_df[(county_df['state'] == STATE) & (county_df['county'] == COUNTY)])

print(country_df.tail())
print(state_df.tail())
print(county_df.tail())

In [ ]:
SIZE = (20, 12)

fig, axs = plt.subplots(3, 2, figsize=SIZE)


def draw_daily_graphs(ax_row, df, region_label):
    ax_row[0].set(xticks=[], title=f'Daily Cases {region_label}')
    ax_row[0].bar(df.index, df['daily_cases'])
    ax_row[0].plot(df.index, df['daily_cases'].rolling(7, 1).mean(), color='red')
    
    ax_row[1].set(xticks=[], title=f'Daily Deaths {region_label}')
    ax_row[1].bar(df.index, df['daily_deaths'])
    ax_row[1].plot(df.index, df['daily_deaths'].rolling(7, 1).mean(), color='red')


draw_daily_graphs(axs[0], country_df, 'U.S.')
draw_daily_graphs(axs[1], state_df, f'State: {STATE}')
draw_daily_graphs(axs[2], county_df, f'County: {COUNTY}')

plt.show()

In [ ]:
US_POPULATION = 327_167_439
STATE_POPULATION = 39_557_045
COUNTY_POPULATION = 3_185_968


def per_million(cases, deaths, population):
    df = pd.DataFrame()
    df['cases'] = 1_000_000 * cases / population
    df['deaths'] = 1_000_000 * deaths / population
    return df


country_per_million_df = per_million(country_df['cases'], country_df['deaths'], US_POPULATION)
state_per_million_df = per_million(state_df['cases'], state_df['deaths'], STATE_POPULATION)
county_per_million_df = per_million(county_df['cases'], county_df['deaths'], COUNTY_POPULATION)

print(country_per_million_df.tail())
print(state_per_million_df.tail())
print(county_per_million_df.tail())

In [ ]:
SIZE = (20, 8)

fig, axs = plt.subplots(1, 2, figsize=SIZE)


def draw_per_million_graphs(ax, country_s, state_s, county_s, title):
    ax.set(xticks=[], title=title)
    ax.plot(country_s.index, country_s, color='red', label='U.S.')
    ax.plot(state_s.index, state_s, color='blue', label=f'State: {STATE}')
    ax.plot(county_s.index, county_s, color='green', label=f'County: {COUNTY}')
    ax.legend()


draw_per_million_graphs(
    axs[0],
    country_per_million_df['cases'],
    state_per_million_df['cases'],
    county_per_million_df['cases'],
    'Cases per Million',
)
draw_per_million_graphs(
    axs[1],
    country_per_million_df['deaths'],
    state_per_million_df['deaths'],
    county_per_million_df['deaths'],
    'Deaths per Million',
)

plt.show()